## People Centered COS
## CDOT Data - Investigate Crash Locations 2020 - 2024

Based on a convesation with Zuri on January 8th, she is interested in the following. 

1. Top 10 intersections with crashes
2. Top 10 intersections with fatalities

### Set Up

In [1]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

### Load Data

In [6]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format
from cdot_clean_geography_columns import handle_missing_values_in_geography_columns, \
fill_missing_city_values, create_truncated_lat_long, combine_loc1_loc2_alphabetically 

In [7]:
# Load CDOT Data for a list of years

years = [2020, 2021, 2022, 2023, 2024]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2020.xlsx
['SYSTEM', 'RTE', 'SEC', 'MP', 'DATE']
(86304, 85)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2021.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(97848, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2022.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(95431, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2023.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(102166, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2024.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(99980, 111)


## Prepare Data

### Limit to El Paso County 

In [8]:
print('Loaded Data Size', cdot_pdf_load.shape)

cdot_pdf_elpaso = cdot_pdf_load.loc[cdot_pdf_load['County'] == 'EL PASO']

print('El Paso County Data Size', cdot_pdf_elpaso.shape)

Loaded Data Size (481729, 149)
El Paso County Data Size (49422, 149)


### Clean Data

In [9]:
# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_elpaso.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)

display(cdot_pdf_withNans_dedup.head(5))

Data size after drop duplicates (49417, 149)


,System Code,Crash Date,Injury 00,Injury 01,Injury 02,Injury 03,Injury 04,Agency Id,Location,Road Description,...,Wild Animal,Road Contour Curves,Road Contour Grade,Number Killed,Number Injured,Approach Overtaking Turn,Crash Time,Rd_Number,Rd_Section,City_Street
9,City Street,2020-01-01,1.0,0.0,0.0,0.0,0.0,CSPD,On Roadway,Non-Intersection,...,NaN,Straight,Level,0.0,0.0,Not Applicable,31:00:00,NaN,VERDE,NaN
14,City Street,2020-01-01,1.0,0.0,0.0,0.0,0.0,CSPD,On Roadway,Non-Intersection,...,NaN,Straight,Level,0.0,0.0,Not Applicable,10:50:00,NaN,BOULD,NaN
16,County Road,2020-01-01,1.0,0.0,0.0,0.0,0.0,CSP,Ran off right side,Non-Intersection,...,NaN,Curve Unknown Direction,Unknown,0.0,0.0,Not Applicable,11:50:00,NaN,MOUNT,NaN
18,Interstate Highway,2020-01-01,0.0,0.0,1.0,0.0,0.0,CSP,Ran off left side,Non-Intersection,...,NaN,Straight,Level,0.0,1.0,Not Applicable,12:50:00,25.0A,131.0,NaN
31,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,CSPD,On Roadway,Non-Intersection,...,NaN,Straight,Unknown,0.0,0.0,Not Applicable,30:00:00,NaN,BRIDG,NaN


In [10]:
# Select columns for analysis 
analysis_columns = [
    'Crash Date', 
    'City', 
    'County',
    'Crash Year',
    'Latitude', 
    'Longitude',
    'Location 1', 
    'Location 2',
]
cdot_pdf = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
print('nan count before handling') 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])
print()

# handle nan, null, and other corner cases
cdot_pdf = handle_missing_values_in_geography_columns(cdot_pdf)

# check NaN count by column 
print('nan count after handling')
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])

nan count before handling
Crash Date 0
Crash Date 0
City 10916
City 10916
County 0
County 0
Crash Year 0
Crash Year 0
Latitude 9073
Latitude 9073
Longitude 9073
Longitude 9073
Location 1 1
Location 1 1
Location 2 30
Location 2 30

nan count after handling
Crash Date 0
Crash Date 0
City 0
City 0
County 0
County 0
Crash Year 0
Crash Year 0
Latitude 0
Latitude 0
Longitude 0
Longitude 0
Location 1 1
Location 1 1
Location 2 30
Location 2 30


In [11]:
# SKIP for RUNTIME - fill missing city values 
#cdot_pdf_select_columns = fill_missing_city_values(cdot_pdf_select_columns)
# display(cdot_pdf.head(10))

In [12]:
# Create truncated latitude and longitude for analysis 
cdot_pdf = create_truncated_lat_long(cdot_pdf, 3) 
print(cdot_pdf.columns)

Index(['Crash Date', 'City', 'County', 'Crash Year', 'Latitude', 'Longitude',
       'Location 1', 'Location 2', 'Latitude_3dec', 'Longitude_3dec'],
      dtype='object')


In [13]:
# combine location 1 and location 2
# more useful to understand the intersection vs. location 1 at location 2 being different than location 2 at location 2. 
# *************************
cdot_pdf = combine_loc1_loc2_alphabetically(cdot_pdf) 

# check result
display(cdot_pdf[['Location 1', 'Location 2', 'loc1_first', 'alph_loc1_loc2']].head(5))

,Location 1,Location 2,loc1_first,alph_loc1_loc2
9,VERDE DR,CHELTON RD S,False,CHELTON RD S VERDE DR
14,BOULDER ST E,NEVADA AVE N,True,BOULDER ST E NEVADA AVE N
16,MOUNTAIN PINE LN,ROLLER COASTER RD,True,MOUNTAIN PINE LN ROLLER COASTER RD
18,I-25,MM 131,True,I-25 MM 131
31,BRIDGEWOOD LN,CHELTON RD S,True,BRIDGEWOOD LN CHELTON RD S


## Analyze Data - Count Crashes Geographically 

- First priority is Lat Long
- Second priority is street names

### Top Crash Intersections for All Analysis Years

In [14]:
# Get the counts of crashes by latitude and longitude truncated values 
n = 15
print(f"Top {n} number of crashes in {years}")
print('Count unique truncated lat long values in dataset', 
      cdot_pdf.groupby(['Latitude_3dec', 'Longitude_3dec']).count()['City'].sort_values(ascending=False).shape[0])

# Get the lat long combos with the highest number of crashes 
top_n_latlong = pd.DataFrame(cdot_pdf.groupby(['Latitude_3dec', 'Longitude_3dec']).count()['City'].sort_values(ascending=False).head(n)) \
                    .reset_index().rename(columns={'City': 'crash_count'})
display(top_n_latlong)

# Get location information and count other crashes with same location strings 
for index, row in pd.DataFrame(top_n_latlong).iterrows():
    print()
    if row['Latitude_3dec'] != 999.999:
        # get additional count of crashes for crashes without lat-long but with the same location 1 and 2 
        associated_loc1_loc2_values = cdot_pdf.loc[(cdot_pdf['Latitude_3dec'] == row['Latitude_3dec']) & 
                     (cdot_pdf['Longitude_3dec'] == row['Longitude_3dec']), 'alph_loc1_loc2'].unique()
        extra_crashes_from_missing_ll_and_same_str = cdot_pdf.loc[(cdot_pdf['alph_loc1_loc2'].isin(associated_loc1_loc2_values)) &
                     (cdot_pdf['Latitude_3dec'] == 999.999)].shape[0]

        print(f"Top {index} number of crashes in {years}")
        print(f"Location:")
        print(f"   Associated lat {row['Latitude_3dec']}")
        print(f"   Associated long {row['Longitude_3dec']}")
        print('   first few location strings', associated_loc1_loc2_values[0:3])
        print('Crash Count:')
        print('   crash count from lat long', row['crash_count'])
        print('   additional crash count from location (associated with lat long):', extra_crashes_from_missing_ll_and_same_str)

Top 15 number of crashes in [2020, 2021, 2022, 2023, 2024]
Count unique truncated lat long values in dataset 10140


,Latitude_3dec,Longitude_3dec,crash_count
0,999.999,999.999,10123
1,38.815,-104.822,216
2,38.911,-104.720,185
3,38.719,-104.701,178
4,38.828,-104.835,161
5,38.894,-104.720,157
6,38.897,-104.830,155
7,38.846,-104.721,152
8,38.926,-104.720,128
9,38.825,-104.725,123




Top 1 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.815
   Associated long -104.822
   first few location strings ['I-25 NB OFF RAMP NEVADA AVE S' 'I-25 NB ON RAMP NEVADA AVE S'
 'I-25 SB ON RAMP NEVADA AVE S']
Crash Count:
   crash count from lat long 216.0
   additional crash count from location (associated with lat long): 3

Top 2 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.911
   Associated long -104.72
   first few location strings ['POWERS BLVD N STETSON HILLS BLVD' 'POWERS B N STETSON HILLS B'
 'POWERS BLVD N STETSON HILLS B']
Crash Count:
   crash count from lat long 185.0
   additional crash count from location (associated with lat long): 8

Top 3 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.719
   Associated long -104.701
   first few location strings ['FOUNTAIN MESA RD MESA RIDGE PKWY' 'FOUNTAIN MESA R MESA RIDGE PK'
 'FOUNTAIN MESA RD MESA RIDGE PKWY WB']
Cras

### Counts by Lat Long by Year

In [16]:
# Get the lat long combos with the highest number of crashes 
gb_ct = cdot_pdf.groupby(['Crash Year', 'Latitude_3dec', 'Longitude_3dec']).count()['City'].reset_index().rename(columns={'City': 'crash_count'})
#display(gb_ct)

crash_ct_latlong_by_year = gb_ct.pivot_table(index=['Latitude_3dec', 'Longitude_3dec'], columns='Crash Year', values='crash_count', fill_value=0)
crash_ct_latlong_by_year.columns = [f'Year {col} Count' for col in crash_ct_latlong_by_year.columns]
crash_ct_latlong_by_year = crash_ct_latlong_by_year.reset_index() # Bring Lat and Long back as columns
#display(crash_ct_latlong_by_year)
display(crash_ct_latlong_by_year.loc[crash_ct_latlong_by_year['Year 2024.0 Count'] > 20].sort_values('Year 2024.0 Count', ascending=False))

,Latitude_3dec,Longitude_3dec,Year 2020.0 Count,Year 2021.0 Count,Year 2022.0 Count,Year 2023.0 Count,Year 2024.0 Count
2886,38.815,-104.822,30.0,44.0,45.0,39.0,58.0
1769,38.772,-104.758,0.0,1.0,1.0,5.0,39.0
6492,38.897,-104.830,38.0,31.0,24.0,33.0,29.0
4076,38.840,-104.776,21.0,9.0,6.0,14.0,29.0
4100,38.840,-104.748,22.0,14.0,14.0,32.0,28.0
6290,38.891,-104.757,34.0,11.0,10.0,37.0,28.0
7539,38.933,-104.813,25.0,25.0,13.0,22.0,27.0
10139,999.999,999.999,920.0,4633.0,3835.0,708.0,27.0
2883,38.815,-104.825,18.0,32.0,16.0,20.0,27.0
5021,38.857,-104.720,5.0,14.0,29.0,25.0,25.0
